In [1]:
import pyspark  
sc = pyspark.SparkContext('local[*]') 
# do something to prove it works 
rdd = sc.parallelize(range(1000)) 
rdd.takeSample(False, 5) 

[400, 329, 43, 34, 557]

In [3]:
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV Example") \
    .getOrCreate()
# accounts
df_clean_file = spark.read \
    .option("header", "true")  \
    .option("inferSchema", "true")  \
    .csv("clean_me.csv", sep=";")

df_clean_file.count()

499999

Write a function to format the file in a way that each appropriate value will correspond to only one column. If the data itself contains the splitter comma, it should be quoted by double quotes.  

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, split, concat, when

import pandas as pd  # Note: This import is not used in the provided code. Consider removing it.

def format_file():
    """
    Read a CSV file, format its content and return a DataFrame with the desired columns.
    """
    # Read the text file
    raw_file = spark.read.text("clean_me.csv")
    
    # Splitting the columns
    formatted_df = raw_file.withColumn("splitted_column", split(col("value"), ","))
    
    # Extracting fields from the splitted_column
    formatted_df = (formatted_df
                    .withColumn("order_id", col("splitted_column")[0])
                    .withColumn("delivery_company", col("splitted_column")[1])
                    .withColumn("quantity", when(col("splitted_column")[2].cast("int").isNotNull(), col("splitted_column")[2])
                                .otherwise(lit(0)))
                    .withColumn("price", when(col("splitted_column")[4].cast("int").isNotNull(),
                                               concat(col("splitted_column")[3], lit("."), col("splitted_column")[4]))
                                .otherwise(col("splitted_column")[3]))
                    .withColumn("ordered_date", when(col("splitted_column")[4].cast("int").isNotNull(), col("splitted_column")[5])
                                .otherwise(col("splitted_column")[4]))
                    .withColumn("address", concat(col("splitted_column")[6], lit(","), col("splitted_column")[7]))
                    )
    
    # Removing the header and selecting desired columns
    result_df = (formatted_df.select("order_id", "delivery_company", "quantity", "price", "ordered_date", "address")
                 .filter(col("order_id") != "order_id"))
    
    return result_df

# Creating a SparkSession
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Calling the function and showing the result
format_file().show()
